<a href="https://colab.research.google.com/github/GuShanzhi/Chinese-alpaca-lora/blob/main/share/evaluation_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Apr  4 03:26:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.6 MB/s eta 0:00:00
  Installing b

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "silk-road/luotuo-lora-7b-1.0")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
import functools
import time

def log_time(func):
    @functools.wraps(func)
    def timer(*args, **kwargs):
        func_name = func.__qualname__
        start_time = time.time()
        result = func(*args, **kwargs)
        cost_time = time.time() - start_time
        cost_minute = cost_time // 60
        cost_second = cost_time - (cost_minute * 60)
        print(f"{func_name} costs time: {int(cost_minute)}m {cost_second:.2f}s\n")
        return result
    return timer

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

@log_time
def evaluate(instruction, input=None):
    print(f"Instruction: {instruction}")
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [ ]:
sentence_list = [
    "网元是什么意思",
    "帮我生成一条关于春游的团建宣传标语",
    "我要你帮我创建一个割接核查任务：我会给你输入一批网元，你要帮我检查输入的网元是否有告警，链路是否正常，端口的光功率是否异常，网元上的业务是否有告警。请你列出需要检查的诊断项以及任务步骤"
]

for sentence in sentence_list:
    evaluate(sentence)

Instruction: 网元是什么意思
Response: 网元是一种网络技术，它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建立连接。它可以帮助网络设备与其他网络设备之间建
evaluate costs time: 1m 20.90s

Instruction: 帮我生成一条关于春游的团建宣传标语
Response: 欢迎参加我们的春游！
evaluate costs time: 0m 11.94s

Instruction: 我要你帮我创建一个割接核查任务：我会给你输入一批网元，你要帮我检查输入的网元是否有告警，链路是否正常，端口的光功率是否异常，网元上的业务是否有告警。请你列出需要检查的诊断项以及任务步骤
Response: 1. 检查网元是否有告警
2. 检查网元的光功率是否异常
3. 检查端口的光功率是否异常
4. 检查网元上的业务是否有告警

步骤：
1. 检查网元是否有告警
2. 检查网元的光功率是否异常
3. 检查端口的光功率是否异常
4. 检查网元上的业务是否有告警
evaluate costs time: 1m 11.53s



In [ ]:
evaluate(input("Instruction: "))

Instruction: 如何实现代码可信
Instruction: 如何实现代码可信
Response: 实现代码可信的方法包括：
1. 使用可靠的编译器来编写代码；
2. 使用可靠的编译器来编译代码；
3. 使用可靠的编译器来检查代码；
4. 使用可靠的测试工具来测试代码；
5. 使用可靠的测试工具来检查代码；
6. 使用可靠的测试工具来检查代码；
7. 使用可靠的测试工具来检查代码；
8. 使用可靠的测试工具来检查代码；
9. 使用可靠的测试工具来检查代码；
10. 使用可靠的测试工具来检查
evaluate costs time: 1m 7.31s

